In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statistics as stat

import os
import logging
import math
import itertools
import functools

import scipy.stats
import typing
import random
import time
from datetime import datetime

import tensorflow as tf

logging.basicConfig(level=logging.DEBUG)

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
DEBUG:matplotlib.backends:backend module://ipykernel.pylab.backend_inline version unknown


In [2]:
# If we are running inside Colaboratory, fetches other files from git and chdir into it
if not os.path.exists("dataset"):
    logging.warning(f'Fetching files from GitHub')
    
    !git clone --recurse-submodules https://github.com/paulo-raca/mo810-stock-predict.git workdir
    os.chdir('workdir')
    ! > is_colab
    
is_colab = os.path.exists('is_colab')

import softops
import tensorboard
import stock_dataset
from normalize import log_perc

In [3]:
def debug(x, msg):
    return tf.Print(x, [x], msg)       

In [4]:
class TradingState:
    def __init__(self, money: tf.Tensor, stocks: tf.Tensor, stock_prices: tf.Tensor, inner_state: typing.Any = None, money_value: tf.Tensor = None, stocks_value: tf.Tensor = None, account_value: tf.Tensor = None):
        self.money = tf.identity(money, name='money')
        self.stocks = tf.identity(stocks, name='stocks')
        self.stock_prices = tf.identity(stock_prices, name='stock_prices')
        self.inner_state = inner_state

        if money_value is not None:
            self.money_value = tf.identity(money_value, name='money_value')
        else:
            self.money_value = tf.reduce_sum(self.money, axis=1, name='money_value')
            
        if stocks_value is not None:
            self.stocks_value = tf.identity(stocks_value, name='stocks_value')
        else:
            self.stocks_value = tf.reduce_sum(self.stocks*self.stock_prices, axis=1, name='stocks_value')
            
        if account_value is not None:
            self.account_value = tf.identity(account_value, name='account_value')
        else:
            self.account_value = tf.add(self.money_value, self.stocks_value, name='account_value')

    def copy(self, **kwargs):
        args = dict(
            money = self.money,
            stocks = self.stocks,
            stock_prices = self.stock_prices,
            inner_state = self.inner_state,
            money_value = self.money_value,
            stocks_value = self.stocks_value,
            account_value = self.account_value)
        args.update(kwargs)
        return TradingState(**args)

In [5]:
FEATURE_LOW    = 0
FEATURE_HIGH   = 1
FEATURE_OPEN   = 2
FEATURE_CLOSE  = 3
FEATURE_VOLUME = 4
NUM_FEATURES   = 5

RESPONSE_SELL_LOW    = 0
RESPONSE_SELL_HIGH   = 1
RESPONSE_BUY_PRICE   = 2
RESPONSE_BUY_AMOUNT  = 3
NUM_RESPONSES        = 4

MONEY_TODAY = 0
MONEY_TOMORROW = 1

SELL_TRANSACTION_COST = 5 # 5 USD per transaction
BUY_TRANSACTION_COST = 5  # 5 USD per transaction

def trader_builder(minibatch_size, num_companies, historic_window_size, future_window_size, trainable_variables_node):    
    num_output_responses = NUM_RESPONSES
    num_output_predictions = future_window_size*NUM_FEATURES
    num_outputs = num_output_responses + num_output_predictions

    lstm_sizes = [30, num_outputs]
        
    with trainable_variables_node:
        multi_lstm_cell = tf.nn.rnn_cell.MultiRNNCell([
            tf.nn.rnn_cell.GRUCell(lstm_size)
            for lstm_size in lstm_sizes
        ], state_is_tuple=True)
        
        output_bias = tf.Variable(tf.zeros([num_outputs]), name='output_bias')
        output_scale = tf.Variable(tf.ones([num_outputs]), name='output_scale')
        
        
        
    first_inner_state = multi_lstm_cell.zero_state(minibatch_size*num_companies, tf.float32)
        
    def build_trader(state: TradingState, historic_data: tf.Tensor, inner_state: typing.Any) -> tf.Tensor:
        """
        Builds the trader network on tensorflow

        Arguments:
        - state: Trading state coming from the previous day
        - historic_data: tensor[minibatch, company, time] -> [low, high, open, close]

        Returns: 
        tensor[minibatch, company] -> [buy_price, buy_amount, sell_low_price, sell_high_price]
        - buy_price: The price for which you want to buy stocks
        - buy_amount: Fraction of the cash available to be used to buy stocks from this company (If the price target is reached)
        - sell_high_price: If the stock price reaches more than this amount, all of the owned stocks will be sold (Ideal case, sell when the price is high)
        - sell_low_price: If the stock price reaches less than this amount, all of the owned stocks will also be sold (Fucked up case: Prices are crashing, minimize losses)
        """
        with tf.name_scope('normalize_input'):
            historic_data_shape = tf.shape(historic_data)
            normalize_wrt = tf.stack(
                [
                    historic_data[:, :, -1, FEATURE_CLOSE],
                    historic_data[:, :, -1, FEATURE_CLOSE],
                    historic_data[:, :, -1, FEATURE_CLOSE],
                    historic_data[:, :, -1, FEATURE_CLOSE],
                    tf.ones([historic_data_shape[0], historic_data_shape[1]]),#historic_data[:, :, -1, FEATURE_VOLUME],
                ],
                axis = 2
            )
            normalize_wrt = tf.tile(normalize_wrt[:, :, tf.newaxis, :], [1, 1, historic_window_size, 1], name='normalize_wrt')
            
            normalized_history = log_perc.normalize(
                historic_data,
                normalize_wrt,
                name='normalized_history'
            )
            
            normalized_features = tf.reshape(
                normalized_history,
                [
                    historic_data_shape[0]*historic_data_shape[1],
                    historic_window_size*NUM_FEATURES
                ])
            
            #tf.summary.histogram('input_features', tf.clip_by_value(normalized_features, -5, +5))
        
        with tf.name_scope('prediction'):
            with tf.name_scope('RNN'):
                outputs, inner_state = multi_lstm_cell(normalized_features, inner_state)
            outputs = outputs * output_scale + output_bias
            
            trade_decisions = tf.reshape(outputs[:, :num_output_responses], [historic_data_shape[0], historic_data_shape[1], num_output_responses], name="decisions")
            predictions     = tf.reshape(outputs[:, num_output_responses:], [historic_data_shape[0], historic_data_shape[1], future_window_size, NUM_FEATURES], name="predictions")
        
        with tf.name_scope('buy_amount'):
            buy_amount = tf.fill([minibatch_size, num_companies], value=0.1)
            
        with tf.name_scope('denormalize_prices'):
            reference_price = historic_data[:, :, -1, FEATURE_CLOSE]
            reference_price = tf.tile(reference_price[:, :, tf.newaxis], [1, 1, num_output_responses])
            
            #tf.summary.histogram('predictions', predictions)
            
            trade_decisions = log_perc.denormalize(
                trade_decisions,
                reference_price,
                'trade_decisions')
            predictions = log_perc.denormalize(
                predictions,
                normalize_wrt,
                'predictions')
            
            #sell_low_price = log_perc.denormalize(trade_decisions[:, :, RESPONSE_SELL_LOW], reference_price,'sell_low_price')
            #sell_high_price = log_perc.denormalize(trade_decisions[:, :, RESPONSE_SELL_HIGH], reference_price, 'sell_high_price')
            #buy_price = log_perc.denormalize(trade_decisions[:, :, RESPONSE_BUY_PRICE], reference_price,'buy_price')
            
        with tf.name_scope('result'):
            return (trade_decisions, predictions, inner_state)
        
    return build_trader, first_inner_state

In [6]:
def build_env_step(build_trader, state: TradingState, historic_data: tf.Tensor, future_data: tf.Tensor) -> TradingState:
    """
    Builds a single-day trading environment.
    Arguments:
    - state: Trading state coming from the previous day
    - historic_data: tensor[minibatch, company, time] -> [low, high, open, close]
    - next_day_data: tensor[minibatch, company] -> [low, high, open, close]

    
    Returns: The next state, computed at the end of the next trading day
    """   
    with tf.name_scope("trader"):
        trade_decisions, trade_predictions, new_inner_state = build_trader(state = state, historic_data = historic_data, inner_state = state.inner_state)
       
        
    with tf.name_scope("simulator"):
        softness = 0.1
        
        with tf.name_scope("slicing"):
            sell_low_price  = trade_decisions[:, :, RESPONSE_SELL_LOW]
            sell_high_price = trade_decisions[:, :, RESPONSE_SELL_HIGH]
            buy_price       = trade_decisions[:, :, RESPONSE_BUY_PRICE]
            buy_amount      = tf.fill(tf.shape(buy_price), value=0.1) #trader[:, :, RESPONSE_BUY_AMOUNT]
            
            current_money   = state.money[:, MONEY_TODAY]
            current_stocks  = state.stocks
            next_day_data = future_data[:, :, 0, :]
            eod_stock_prices = next_day_data[:, :, FEATURE_CLOSE]

        with tf.name_scope("predictions"):
            predictions_error  = log_perc.normalize(
                trade_predictions[:, :, :, :FEATURE_VOLUME],
                future_data[:, :, :, :FEATURE_VOLUME],
                'percent_error')
            
        # Executes SELL transactions if prices reaches BELOW a threshold (minimize losses)            
        # (If the day opens below the threshold for LOW SELL, use the open price)
        with tf.name_scope("sell_low"):
            sell_low_price_error  = log_perc.normalize(
                debug(sell_low_price, "sell_low_price"),
                debug(tf.reduce_min(future_data[:, :, :, FEATURE_LOW], axis=2), "FEATURE_LOW"),
                'price_error')
            
            sell_low_price = tf.identity(
                sell_low_price,
                name='order_price')
            sell_low_price = tf.minimum(
                next_day_data[:, :, FEATURE_OPEN],
                sell_low_price, 
                name = 'actual_price')
            sell_low_amount = tf.identity(
                current_stocks,
                name = 'order_amount')
            sell_low_kernel = 0*tf.multiply(
                softops.gte(sell_low_price, next_day_data[:, :, FEATURE_LOW], percent = True, softness = softness),
                softops.positive(sell_low_amount),
                name = 'order_executed')
                
            sell_low_stock_bough = tf.zeros(tf.shape(sell_low_kernel), name='stock_bought')
            sell_low_stock_sold = tf.multiply(sell_low_kernel, sell_low_amount, name='stock_sold')
            money_earned_sell_low = tf.reduce_sum(sell_low_stock_sold * sell_low_price, axis=1, name='money_earned')
            money_spent_sell_low = tf.reduce_sum(sell_low_kernel * SELL_TRANSACTION_COST, axis=1, name='money_spent')

        # Executes SELL transactions if prices reaches ABOVE a threshold (Maximize gains)
        with tf.name_scope("sell_high"):
            sell_high_price_error = log_perc.normalize(
                debug(sell_high_price, "sell_high_price"),
                debug(tf.reduce_max(future_data[:, :, :, FEATURE_HIGH], axis=2), "FEATURE_HIGH"),
                'price_error')
            
            sell_high_price = tf.identity(
                sell_high_price,
                name='order_price')
            sell_high_price = tf.identity(
                sell_high_price,
                name='actual_price')
            sell_high_amount = tf.identity(
                current_stocks - sell_low_stock_sold,
                name = 'order_amount')
            sell_high_kernel = tf.multiply(
                softops.lte(sell_high_price, next_day_data[:, :, FEATURE_HIGH], percent = True, softness = softness),
                softops.positive(sell_high_amount),
                name = 'order_executed')
                
            sell_high_stock_bough = tf.zeros(tf.shape(sell_high_kernel), name='stock_bought')
            sell_high_stock_sold = tf.multiply(sell_high_kernel, sell_high_amount, name='stock_sold')
            money_earned_sell_high = tf.reduce_sum(sell_high_stock_sold * sell_high_price, axis=1, name='money_earned')
            money_spent_sell_high = tf.reduce_sum(sell_high_kernel * SELL_TRANSACTION_COST, axis=1, name='money_spent')

        # Executes BUY transactions if prices reaches above a threshold
        # (Normalize `buy_amount` to be in amount of stocks, instead of fraction of my money)
        with tf.name_scope("buy"):
            buy_price_error = log_perc.normalize(
                debug(buy_price, "buy price"),
                debug(next_day_data[:, :, FEATURE_LOW], "FEATURE_LOW"),
                'price_error')
            
            buy_price = tf.identity(
                buy_price,
                name = 'order_price')
            buy_price = tf.identity(
                buy_price,
                name = 'actual_price')
            buy_amount = softops.floor(
                buy_amount * current_money[:, tf.newaxis] / buy_price,
                name = 'order_amount')
            buy_kernel = tf.multiply(
                softops.gte(buy_price, next_day_data[:, :, FEATURE_LOW], percent = True, softness = softness),
                softops.positive(buy_amount),
                name='kernel')
                
            buy_stock_bough = tf.multiply(buy_kernel, buy_amount, name='stock_sold')
            buy_stock_sold = tf.zeros(tf.shape(buy_kernel), name='stock_bought')
            money_earned_buy = tf.zeros(tf.shape(current_money), name='money_earned')
            money_spent_buy = tf.reduce_sum(buy_stock_bough * buy_price + buy_kernel * BUY_TRANSACTION_COST, axis=1, name='money_spent')

        with tf.name_scope("update_money"):           
            total_money_earned = money_earned_sell_high + money_earned_sell_low + money_earned_buy
            total_money_spent = money_spent_sell_low + money_spent_sell_high + money_spent_buy
            new_money_first = tf.expand_dims(current_money - total_money_spent + state.money[:, MONEY_TOMORROW], axis = 1)
            new_money_middle = state.money[:, 2:]
            new_money_last = tf.expand_dims(total_money_earned, axis = 1)
            
            next_money = tf.concat([
                new_money_first,
                new_money_middle,
                new_money_last
            ], axis = 1)

#             with tf.name_scope("money_earned"):
#                 tf.summary.histogram('values', total_money_earned)
#                 tf.summary.scalar('mean', tf.reduce_mean(total_money_earned))
#             with tf.name_scope("money_spent"):
#                 tf.summary.histogram('values', total_money_spent)
#                 tf.summary.scalar('mean', tf.reduce_mean(total_money_spent))
#             with tf.name_scope("money_delta"):
#                 tf.summary.histogram('values', total_money_earned - total_money_spent)
#                 tf.summary.scalar('mean', tf.reduce_mean(total_money_earned - total_money_spent))
#             with tf.name_scope("money_final"):
#                 tf.summary.histogram('values', tf.reduce_sum(next_money, axis=1))
#                 tf.summary.scalar('mean', tf.reduce_mean(tf.reduce_sum(next_money, axis=1)))

        with tf.name_scope("update_stocks"):
            total_stocks_bought = sell_low_stock_bough + sell_high_stock_bough + buy_stock_bough
            total_stocks_sold = sell_low_stock_sold + sell_high_stock_sold + buy_stock_sold
            next_stocks = current_stocks + total_stocks_bought - total_stocks_sold
            next_stocks_value = tf.reduce_sum(next_stocks * eod_stock_prices, axis=1)
            
#             with tf.name_scope("stocks_bought"):
#                 tf.summary.histogram('values', total_stocks_bought)
#                 tf.summary.scalar('mean', tf.reduce_mean(total_stocks_bought))
#             with tf.name_scope("stocks_sold"):
#                 tf.summary.histogram('values', total_stocks_sold)
#                 tf.summary.scalar('mean', tf.reduce_mean(total_stocks_sold))
#             with tf.name_scope("stocks_delta"):
#                 tf.summary.histogram('values', total_stocks_bought - total_stocks_sold)
#                 tf.summary.scalar('mean', tf.reduce_mean(total_stocks_bought - total_stocks_sold))
#             with tf.name_scope("stocks_final"):
#                 tf.summary.histogram('values', next_stocks)
#                 tf.summary.scalar('mean', tf.reduce_mean(next_stocks))
#             with tf.name_scope("stocks_value_final"):
#                 tf.summary.histogram('values', next_stocks_value)
#                 tf.summary.scalar('mean', tf.reduce_mean(next_stocks_value))
#             with tf.name_scope("stocks_value_delta"):
#                 tf.summary.histogram('values', next_stocks_value - state.stocks_value)
#                 tf.summary.scalar('mean', tf.reduce_mean(next_stocks_value - state.stocks_value))
    
        return dict(
            money = next_money,
            stocks = next_stocks,
            stock_prices = eod_stock_prices,
            stocks_value= next_stocks_value,
            inner_state = new_inner_state
        )
        

In [7]:
def build_env(trainable_variables_node, historic_data, initial_state, num_days, historic_window_size, future_window_size):    
    build_trader, first_inner_state = trader_builder(
        minibatch_size = tf.shape(historic_data)[0],
        num_companies = tf.shape(historic_data)[1],
        historic_window_size = historic_window_size,
        future_window_size = future_window_size,
        trainable_variables_node = trainable_variables_node)
    
    with tf.name_scope(f'state_0'):
        current_state = TradingState(inner_state = first_inner_state, **initial_state)
    all_states = [current_state]
    
        
    for i in range(num_days):
        #print(f'day_{i}')
        with tf.name_scope(f'day_{i}'):
            with tf.name_scope(f'historic_data'):
                historic_slice = historic_data[:, :, i:i+historic_window_size, :]
            with tf.name_scope(f'future_data'):
                future_slice = historic_data[:, :, i+historic_window_size:i+historic_window_size+future_window_size, :]

            current_state = build_env_step(
                build_trader = build_trader,
                state = current_state,
                historic_data = historic_slice,
                future_data = future_slice
            )
            
        with tf.name_scope(f'state_{i+1}'):
            current_state = TradingState(**current_state)

        all_states.append(current_state)
    return all_states

In [8]:
def build_graph(warmup_days = 10, evaluated_days = 20, historic_window_size = 5, future_window_size = 5, money_settle_time = 3, check_numerics = False) -> tf.Graph:
    total_days = evaluated_days + warmup_days + historic_window_size + future_window_size - 1
    
    graph = tf.Graph()
    with graph.as_default():        
        trainable_variables_node = tf.name_scope("trainable_parameters")
        iteration = tf.Variable(0, name='iteration', dtype=tf.int32, expected_shape=())
                    
        with tf.name_scope("inputs"):
            stock_history = tf.placeholder(tf.float32, shape=(None, None, total_days, 5), name="stock_history")

            minibatch_size = tf.shape(stock_history)[0]
            num_companies = tf.shape(stock_history)[1]
            default_initial_stocks = tf.zeros([minibatch_size, num_companies], name = 'default_initial_stocks')
            initial_stocks = tf.placeholder_with_default(default_initial_stocks, shape = (None, None), name = 'initial_stocks')

            initial_money_simple = tf.placeholder_with_default(100000., shape = (), name = 'initial_money_simple')
            default_initial_money = tf.concat(
                [
                    initial_money_simple * tf.ones([minibatch_size, 1]),
                    tf.zeros([minibatch_size, money_settle_time-1])
                ], 
                axis = 1,
                name = 'default_initial_money')
            initial_money = tf.placeholder_with_default(default_initial_money, shape = (None, money_settle_time), name = 'initial_money')
            
            with tf.name_scope("initial_state"):
                initial_state = dict(
                    money = initial_money,
                    stocks = initial_stocks,
                    stock_prices = stock_history[:, :, historic_window_size - 1, FEATURE_CLOSE],
                )

        with tf.name_scope("trading"):
            all_states = build_env(trainable_variables_node, stock_history, initial_state, warmup_days + evaluated_days, historic_window_size, future_window_size)
            initial_account_value = all_states[0].account_value

        with tf.name_scope("evaluation"):
            def new_summary(values, clip=None, summaries=['mean'], family=None):
                values = tf.identity(values, name='values')
                values_clipped = values
                if clip is not None:
                    values_clipped = tf.clip_by_value(values, clip[0], clip[1], name='values_clipped')

                with tf.name_scope('metrics'):
                    mean, variance = tf.nn.moments(values, axes=None)
                    mean = tf.identity(mean, name='mean')
                    variance = tf.identity(variance, name='variance')
                    stddev = tf.sqrt(variance, name='stddev')
                    mean_sqr = tf.reduce_mean(values*values, name="mean_sqr")
                    sqrt_mean_sqr = tf.sqrt(mean_sqr, name="sqrt_mean_sqr")
                    mean_abs = tf.reduce_mean(tf.abs(values), name="mean_abs")
                    min = tf.reduce_min(values, name="min")
                    max = tf.reduce_max(values, name="max")
                    
                with tf.name_scope('summaries'):
                    scalar_summaries = {
                        'mean': mean,
                        'mean_sqr': mean_sqr,
                        'sqrt_mean_sqr': sqrt_mean_sqr,
                        'mean_abs': mean_abs,
                        'variance': variance,
                        'stddev': stddev,
                        'min': min,
                        'max': max
                    }
                    for summary_name in summaries:
                        tf.summary.scalar(summary_name, scalar_summaries[summary_name], family=family)
                    tf.summary.histogram('values', values_clipped, family=family)
            
            with tf.name_scope("account_values"):
                account_values = tf.stack([
                    all_states[i].account_value
                    for i in range(warmup_days, warmup_days + evaluated_days + 1)
                ], axis = 1, name = 'values')
                
            with tf.name_scope("daily_variation"):
                daily_variation = log_perc.normalize(account_values[:, 1:], account_values[:, :-1])
                new_summary(daily_variation, clip=(-5, +5))
                
            with tf.name_scope("period_variation"):
                period_variation = log_perc.normalize(account_values[:, -1], account_values[:, 0])
                new_summary(period_variation, clip=(-100, +100))
                
            with tf.name_scope("final_account_value"):
                new_summary(all_states[-1].account_value, clip=(0, 2*initial_account_value))
                    
            with tf.name_scope("final_money_value"):
                new_summary(all_states[-1].money_value, clip=(0, 2*initial_account_value))
                
            with tf.name_scope("final_stocks_value"):
                new_summary(all_states[-1].stocks_value, clip=(0, 2*initial_account_value))
                                
            with tf.name_scope("sell_low_price_error"):
                new_summary(tf.stack([
                    debug(graph.get_tensor_by_name(f'trading/day_{i}/simulator/sell_low/price_error/value:0'), msg=f'sell_low_price_error[{i}]')
                    for i in range(warmup_days, warmup_days + evaluated_days)
                ], axis=2), clip=(-5, 5), summaries=['sqrt_mean_sqr'], family='price_error')
                
            with tf.name_scope("sell_high_price_error"):
                new_summary(tf.stack([
                    debug(graph.get_tensor_by_name(f'trading/day_{i}/simulator/sell_high/price_error/value:0'), msg=f'sell_high_price_error[{i}]')
                    for i in range(warmup_days, warmup_days + evaluated_days)
                ], axis=2), clip=(-5, 5), summaries=['sqrt_mean_sqr'], family='price_error')
                
            with tf.name_scope("buy_price_error"):
                new_summary(tf.stack([
                    debug(graph.get_tensor_by_name(f'trading/day_{i}/simulator/buy/price_error/value:0'), msg=f'buy_price_error[{i}]')
                    for i in range(warmup_days, warmup_days + evaluated_days)
                ], axis=2), clip=(-5, 5), summaries=['sqrt_mean_sqr'], family='price_error')
                
            with tf.name_scope("all_price_errors"):
                new_summary(tf.stack([
                    graph.get_tensor_by_name(f'evaluation/sell_low_price_error/values:0'),
                    graph.get_tensor_by_name(f'evaluation/sell_high_price_error/values:0'),
                    graph.get_tensor_by_name(f'evaluation/buy_price_error/values:0'),
                ], axis=3), clip=(-5, 5), summaries=['sqrt_mean_sqr'], family='price_error')
                
            with tf.name_scope("prediction_errors"):
                new_summary(tf.stack([
                    debug(graph.get_tensor_by_name(f'trading/day_{i}/simulator/predictions/percent_error/value:0'), msg=f'buy_price_error[{i}]')
                    for i in range(warmup_days, warmup_days + evaluated_days)
                ], axis=2), clip=(-5, 5), summaries=['sqrt_mean_sqr'], family='prediction_errors')
                
        if check_numerics:
            tf.add_check_numerics_ops()
            
        graph.minibatch_producer = functools.partial(stock_dataset.minibatch_producer, timeseries_length=warmup_days+evaluated_days+historic_window_size+future_window_size-1)
            
    return graph    

In [9]:
# Display a small version of the TF graph for visualization/debug
graph = build_graph(warmup_days=1, evaluated_days=1)
tensorboard.show_graph(graph)

In [10]:
def execute_trader(restore=False, save=False, stop_at=None, optimize_profit=False, optimize_price_error=False, optimize_prediction_error=False, learning_rate=0.1, graph_params={}):
    """
    Builds the Stock Trader graph and executes the optimization loop
    
    Arguments:
    - restore: Loads trainning state from a file
    - save: Saves the trainning state to a file (checkpoints done every 30s)
    - stop_at: Stops when the global iteration counter reaches this value
    - optimize_profit: Try to maximize the mean daily profit
    - optimize_price_error: Minimize the difference between the order buy/sell prices and the min/max prices of the next day (A good starting point for profit optimization)
    - optimize_prediction_error: Optimize the predicted open/close/low/high prices of the next few days. While it's not used anywhere per-se, it can be used to pre-train the deep networks used on the trader to extract relevant features.
    - graph_params: Arguments to build_graph()
    """
    print("Building graph...")
    with build_graph(**graph_params).as_default():
        print("Loading dataset")
        next_minibatch = tf.get_default_graph().minibatch_producer()

        print("Creating directories")
        checkpoint_file = os.path.abspath(f'checkpoint/trader.ckpt')
        tensorboard_dir = os.path.abspath('tensorboard_logs')
        tensorboard_run = datetime.utcnow().strftime("%Y-%m-%d-%H-%M-%S")
        tensorboard_run_dir_train = f'{tensorboard_dir}/{tensorboard_run}_train'
        tensorboard_run_dir_test  = f'{tensorboard_dir}/{tensorboard_run}_test'
        os.makedirs(os.path.dirname(checkpoint_file), exist_ok=True)
        os.makedirs(os.path.dirname(tensorboard_run_dir_train), exist_ok=True)   
        os.makedirs(os.path.dirname(tensorboard_run_dir_test), exist_ok=True)   

        print("Files created")
        input_stock_history = tf.get_default_graph().get_tensor_by_name('inputs/stock_history:0')
        input_initial_money = tf.get_default_graph().get_tensor_by_name('inputs/initial_money_simple:0')

        iteration = tf.get_default_graph().get_tensor_by_name('iteration:0')
        iteration_inc = tf.assign_add(iteration, 1, name='iteration_increment')

        evaluation_daily_profit_mean = tf.get_default_graph().get_tensor_by_name('evaluation/daily_variation/metrics/mean:0')
        evaluation_daily_profit_stddev = tf.get_default_graph().get_tensor_by_name('evaluation/daily_variation/metrics/stddev:0')
        
        evaluation_period_profit_mean = tf.get_default_graph().get_tensor_by_name('evaluation/period_variation/metrics/mean:0')
        evaluation_period_profit_stddev = tf.get_default_graph().get_tensor_by_name('evaluation/period_variation/metrics/stddev:0')

        evaluation_price_error = tf.get_default_graph().get_tensor_by_name('evaluation/all_price_errors/metrics/mean_sqr:0')
        evaluation_price_error_pretty = tf.get_default_graph().get_tensor_by_name('evaluation/all_price_errors/metrics/sqrt_mean_sqr:0')

        evaluation_prediction_error = tf.get_default_graph().get_tensor_by_name('evaluation/prediction_errors/metrics/mean_sqr:0')
        evaluation_prediction_error_pretty = tf.get_default_graph().get_tensor_by_name('evaluation/prediction_errors/metrics/sqrt_mean_sqr:0')
        print("Tensors accessed")

        all_summaries = tf.summary.merge_all()                 
        print("Summaries created")

        optimize_for = 0
        if optimize_profit:
            optimize_for += -evaluation_period_profit_mean
        if optimize_price_error:
            optimize_for += evaluation_price_error
        if optimize_prediction_error:
            optimize_for += evaluation_prediction_error
            
        training_op = tf.train.AdagradOptimizer(learning_rate=learning_rate).minimize(optimize_for)
        print("Optimizer created")

        saver = tf.train.Saver()
        print("Trainning Saver created")

        def run_iteration(train=False):
            tf_ops = [
                iteration if train else iteration_inc,
                evaluation_daily_profit_mean,
                evaluation_daily_profit_stddev,
                evaluation_period_profit_mean,
                evaluation_period_profit_stddev,
                evaluation_price_error_pretty,
                evaluation_prediction_error_pretty,
                all_summaries,
            ]
            if train:
                tf_ops.append(training_op)
                
            tf_params = {
                tf.get_default_graph().get_tensor_by_name('inputs/stock_history:0'): next_minibatch(set='train' if train else 'test', minibatch_size=100 if train else 500, num_companies=10),
            }
            result = session.run(tf_ops, feed_dict=tf_params)
            it = result[0]
            daily_profit_mean = result[1]
            daily_profit_stddev = result[2]
            period_profit_mean = result[3]
            period_profit_stddev = result[4]
            price_error = result[5]
            prediction_error = result[6]
            summaries = result[7]

            print(f'{"TRAIN" if train else "TEST"}: epoch [{it}] -- Profit: {daily_profit_mean:.2f}±{daily_profit_stddev:.2f} %/day or {period_profit_mean:.2f}±{period_profit_stddev:.2f} %/period, Order Price Error: {price_error:.2f}%, Overall prediction error: {prediction_error:.2f}%')
            if train:
                tensorboard_writer_train.add_summary(summaries, it)
            else:
                tensorboard_writer_test.add_summary(summaries, it)
            
            return it

        with tf.summary.FileWriter(tensorboard_run_dir_train, tf.get_default_graph()) as tensorboard_writer_train:
            with tf.summary.FileWriter(tensorboard_run_dir_test, tf.get_default_graph()) as tensorboard_writer_test:
                print("Tensorboard Writers created")

                if is_colab:
                    display(tensorboard.Server.of(tensorboard_dir).badge(run=tensorboard_run))

                with tf.Session() as session:
                    print("Session Created")

                    tf.global_variables_initializer().run()
                    print("Initializers executed")

                    try:
                        # Continue a previous trainning session
                        if restore:
                            saver.restore(session, checkpoint_file)
                    except:
                        logging.warning(f'Failed to restore training state from {checkpoint_file}')
                        raise

                    def test_and_save():
                            if save:
                                saver.save(session, checkpoint_file)
                            run_iteration() # Run and log results on test set

                    test_and_save()
                    last_checkpoint = time.time()
                    if optimize_profit or optimize_price_error or optimize_prediction_error:
                        while True:
                            if (time.time() - last_checkpoint) >= 60:
                                last_checkpoint = time.time()
                                test_and_save()

                            it = run_iteration(train=True)

                            if stop_at is not None and it >= stop_at:
                                test_and_save()
                                break

In [11]:
# Initialize as a regression, and try to match buy/sell prices with the min/max of the day
execute_trader(restore=True, save=False, stop_at=10000, optimize_price_error=True, optimize_prediction_error=True)

# Now the we (hopefully) have a decent initial state, proceed with a profit optimization
#execute_trader(restore=True,  save=False, optimize_profit=True)

Building graph...
Loading dataset


Creating directories
Files created
Tensors accessed
Summaries created
Optimizer created
Trainning Saver created
Tensorboard Writers created
Session Created
Initializers executed
INFO:tensorflow:Restoring parameters from /home/jovyan/work/MO810/stock-predict/checkpoint/trader.ckpt


INFO:tensorflow:Restoring parameters from /home/jovyan/work/MO810/stock-predict/checkpoint/trader.ckpt


TEST: epoch [6785] -- Profit: -0.02±0.78 %/day or -0.33±2.83 %/period, Order Price Error: 2.26%, Overall prediction error: 2.82%


KeyboardInterrupt: 

In [ ]:
execute_trader(restore=True, save=False)